<a href="https://colab.research.google.com/github/imbishal7/aops_scraper/blob/main/aops_complete_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import pandas as pd
import numpy as np
import random
from bs4 import BeautifulSoup
import re
import time

In [ ]:
%%shell
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500

Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300

Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF
apt-get update
apt-get install chromium chromium-driver

pip install selenium

Executing: /tmp/apt-key-gpghome.HnwvnmDpOH/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.JM9KmHXsBG/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.O2Dfw3tGpP/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Get:2 http://deb.debian.org/debian bust

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.expected_conditions import presence_of_all_elements_located as presence, all_of
from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

In [ ]:
domain = 'https://artofproblemsolving.com'

In [ ]:
codomains = [('international_contests', '/community/c14_international_contests'),
 ('national_and_regional_contests', '/community/c16_national_and_regional_contests'),
 ('undergraduate_contests', '/community/c15_undergraduate_contests'),
 ('national_olympiads', '/community/c58_national_olympiads'),
 ('team_selection_test', '/community/c59_team_selection_tests'),
 ('junior_olympiads', '/community/c62_junior_olympiads')]

In [ ]:
def driver_setup():
    options = Options()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument("--disable-gpu")
    driver = webdriver.Chrome("/usr/bin/chromedriver", options=options)
    
    return driver

In [ ]:
def scroll_to_end(driver, pause_time=1):
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(pause_time)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

In [ ]:
def get_all_links_from_main_tree(url):
  driver = driver_setup()
  driver.get(url)
  time.sleep(1)
  scroll_to_end(driver)
  
  div_class = ['cmty-category-cell', 'cmty-category-cell-folder', 'cmty-category-cell-no-numrow']

  soup = BeautifulSoup(driver.page_source, 'html.parser')
  divs = soup.find_all('div',{'class': div_class})

  all_links = []
  for i in divs:
    links = i.find('a',{'class':'cmty-full-cell-link'})
    all_links.append(links.get('href'))
  driver.quit()
  return all_links

In [ ]:
def get_all_links_from_second_branch(url):

  driver = driver_setup()
  driver.get(url)
  time.sleep(1)
  scroll_to_end(driver)
  
  sub_contest = 'cmty-category-cell-title'
  div_class = ['cmty-category-cell', 'cmty-category-cell-folder']

  soup = BeautifulSoup(driver.page_source, 'html.parser')

  divs = soup.find_all('div',{'class': div_class})
  title = soup.find('div',{'class':sub_contest}).text

  all_links = []

  for i in divs:
    links = i.find('a',{'class':'cmty-full-cell-link'})
    all_links.append(links.get('href'))
  driver.quit()

  title = title.strip().replace('\n','').replace('\t','')
  return title,all_links

In [ ]:
def get_problems_solutions_link(url):
  name_class = 'cmty-category-cell-title'
  div_class = 'cmty-view-post-poster'
  
  driver = driver_setup()

  wait = WebDriverWait(driver, 10)
  driver.get(url)

  element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, f"div.{div_class}")))
  soup = BeautifulSoup(driver.page_source, 'html.parser')

  name = soup.find('div',{'class': name_class}).text
  name = name.replace('\n', '').replace('\t', '')
  
  links = soup.find_all('div', {'class': div_class})
  only_links = [i.find('a').get('href') for i in links]
  
  driver.quit()
  return name, only_links

In [ ]:
def give_problem_and_all_solutions(problem_url):

  problem_url = problem_url
  
  driver = driver_setup()
  #for problem
  div_class = 'cmty-post-html'
  source_class = "cmty-topic-source-display"
  tag_class = 'cmty-item-tag'
  wait = WebDriverWait(driver, 10)
  driver.get(problem_url)

  element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, f"div.{div_class}")))
  soup = BeautifulSoup(driver.page_source, 'html.parser')

  links = soup.find_all('div', {'class': div_class})

  try:
    problem = links[0].prettify()
  except:
    problem = ''
  try:
    source = soup.find('div', {'class': source_class}).text
  except:
    source = ''
  try:
    tags = soup.find_all('div',{'class': tag_class})
  except:
    tags = ''
  

  info = [source, problem]
  #for solutions
  solutions = []

  soln_post_div_class = 'cmty-post'
  like_count = 'cmty-post-thank-count'
  soln = 'cmty-post-html'
  hidden_soln = 'cmty-hide-content'

  try:
    posts = soup.find_all('div', {'class': soln_post_div_class})
  except:
    post = []

  for i in posts:

    like = i.find('span',{'class': like_count}).text
    try:
      like = int(re.findall(r'\d+', like)[0])
    except:
      like = int(0)


    solution_post = i.find('div',{'class': soln})
    try:
      hidden_post = ''
      hidden_content = solution_post.find_all('div',{'class': hidden_soln})


      for hh in hidden_content:
        hh['style'] = 'display: block;'

        hidden_post += hh.prettify()

      solution=solution_post.prettify() + hidden_post
    except:
      solution = solution_post.prettify()

    soln_with_like = [like,solution]
    if len(str(soln_with_like[1]))> 250:
      solutions.append(soln_with_like)
    else:
      pass

  only_solutions = solutions[1:]
  ranked_solutions = sorted(only_solutions, key=lambda x: x[0], reverse=True)

  no_of_solutions = len(ranked_solutions)
  if no_of_solutions>5:
    ranked_solutions = ranked_solutions[:5]
  else:
    pass
    
  filtered_solutions = [z[1] for z in ranked_solutions]
  return info, ranked_solutions

In [ ]:
main_links = []
for suffix in codomains:
  (contest, end )= suffix
  url = domain + end
  print(f'scraping this link: {contest}')

  links = get_all_links_from_main_tree(url)
  for i in links:
    main_links.append([contest, i])

scraping this link: international_contests


<ipython-input-18-177052b956f3>:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("/usr/bin/chromedriver", options=options)


scraping this link: national_and_regional_contests
scraping this link: undergraduate_contests
scraping this link: national_olympiads
scraping this link: team_selection_test
scraping this link: junior_olympiads


In [ ]:
len(main_links)

303

In [ ]:
main_links_df = pd.DataFrame(data=main_links,columns = ['category', 'link'])

In [ ]:
sub_links_all = []
skipped_main_links = []

for _,row in main_links_df.iloc[:].iterrows():
  contest = row['category']
  main_link_eg = domain+row['link']
  try:
    print(f'scraping this link: {main_link_eg}')
    title, all_links = get_all_links_from_second_branch(main_link_eg)
    print(f'from this link: {main_link_eg}, \n adding {len(all_links)} making the total links: {len(all_links)+len(sub_links_all)}... hold on...')

    for link in all_links:
      sub_links_all.append([contest,title,link])

  except:
    print(f'skipping this link : {main_link_eg}')
    skipped_main_links.append(row['link'])

In [ ]:
len(sub_links_all)

4154

In [ ]:
len(skipped_main_links)

0

In [ ]:
sub_links_all[0]

['international_contests', 'IMO', '/community/c3081085_2022imo_problems']

In [ ]:
all_links_df = pd.DataFrame(data=sub_links_all,columns=['category','contest','link'])

In [ ]:
all_links_df

,Unnamed: 0,category,contest,link
0,0,international_contests,IMO,/community/c3081085_2022imo_problems
1,1,international_contests,IMO,/community/c2413807_2021imo_problems
2,2,international_contests,IMO,/community/c1306546_2020imo
3,3,international_contests,IMO,/community/c912139_2019_imo
4,4,international_contests,IMO,/community/c681585_2018_imo
...,...,...,...,...
4149,4149,junior_olympiads,USAJMO,/community/c3977_2014_usajmo
4150,4150,junior_olympiads,USAJMO,/community/c3976_2013_usajmo
4151,4151,junior_olympiads,USAJMO,/community/c3975_2012_usajmo
4152,4152,junior_olympiads,USAJMO,/community/c3974_2011_usajmo


In [ ]:
all_links_df.to_csv('/content/drive/MyDrive/aops_complete_scraper/all_links_df.csv')

In [ ]:
all_links_df = pd.read_csv('/content/drive/MyDrive/aops_complete_scraper/all_links_df.csv')

In [ ]:
mid_data = []
skipped_mid_data = []

data = all_links_df['link']
for i in data:
  print(f'Progress: {round(data[data==i].index[0]/len(data) * 100, 3)} % and left: {len(data) - data[data==i].index[0]}')
  try:
    name, links = get_problems_solutions_link(domain+i)
    mid_data.append([i,name,links])
  except:
    skipped_mid_data.append(i)
np.save('/content/drive/MyDrive/aops_complete_scraper/mid_data.npy',np.array(mid_data))
np.save('/content/drive/MyDrive/aops_complete_scraper/skipped_mid_data.npy',np.array(skipped_mid_data))

In [ ]:
mid_data = np.load('/content/drive/MyDrive/aops_complete_scraper/mid_data.npy', allow_pickle = True)

In [ ]:
mid_data[0]

array(['/community/c3081085_2022imo_problems', '2022-IMO Problems',
       list(['/community/c6h2883211p25635135', '/community/c6h2883212p25635138', '/community/c6h2883213p25635143', '/community/c6h2883216p25635154', '/community/c6h2883217p25635158', '/community/c6h2883218p25635163'])],
      dtype=object)

In [ ]:
skipped_mid_data = np.load('/content/drive/MyDrive/aops_complete_scraper/skipped_mid_data.npy', allow_pickle = True)

In [ ]:
len(mid_data)

4110

In [ ]:
len(skipped_mid_data)

44

In [ ]:
skipped_mid_data[0]

'/community/c312564_sdmlsdmo_contests'

In [ ]:
new = []
new_skipped = []
data = skipped_mid_data
for i in data:
  try:
    name, links = get_problems_solutions_link(domain+i)
    new.append([i,name,links])
  except:
    new_skipped.append(i)
"""np.save('/content/drive/MyDrive/aops_complete_scraper/mid_data.npy',np.array(mid_data))
np.save('/content/drive/MyDrive/aops_complete_scraper/skipped_mid_data.npy',np.array(skipped_mid_data))"""

<ipython-input-6-177052b956f3>:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("/usr/bin/chromedriver", options=options)


"np.save('/content/drive/MyDrive/aops_complete_scraper/mid_data.npy',np.array(mid_data))\nnp.save('/content/drive/MyDrive/aops_complete_scraper/skipped_mid_data.npy',np.array(skipped_mid_data))"

In [ ]:
len(new), len(new_skipped)

(8, 36)

In [ ]:
total_mid_data = np.append(mid_data, new)

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:1859: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return asanyarray(a).ravel(order=order)


In [ ]:
total_mid_data = mid_data.tolist()

In [ ]:
total_mid_data.extend(new)

In [ ]:
len(total_mid_data)

4118

In [ ]:
total_mid_data[0]

['/community/c3081085_2022imo_problems',
 '2022-IMO Problems',
 ['/community/c6h2883211p25635135',
  '/community/c6h2883212p25635138',
  '/community/c6h2883213p25635143',
  '/community/c6h2883216p25635154',
  '/community/c6h2883217p25635158',
  '/community/c6h2883218p25635163']]

In [ ]:
total_mid_data_df = pd.DataFrame(total_mid_data, columns=['link','name','source'])

In [ ]:
total_mid_data_df.to_csv('/content/drive/MyDrive/aops_complete_scraper/total_mid_data.csv')

In [ ]:
total_mid_data_df

,link,name,source
0,/community/c3081085_2022imo_problems,2022-IMO Problems,"[/community/c6h2883211p25635135, /community/c6..."
1,/community/c2413807_2021imo_problems,2021-IMO Problems,"[/community/c6h2625909p22698392, /community/c6..."
2,/community/c1306546_2020imo,2020-IMO,"[/community/c6h2278651p17821635, /community/c6..."
3,/community/c912139_2019_imo,2019 IMO,"[/community/c6h1876068p12744859, /community/c6..."
4,/community/c681585_2018_imo,2018 IMO,"[/community/c6h1670580p10626500, /community/c6..."
...,...,...,...
4113,/community/c3542_2006_korea__final_round,2006 Korea - Final Round,"[/community/c6h145686p824463, /community/c6h14..."
4114,/community/c1975415_1982_swedish_mathematical_...,1982 Swedish Mathematical Competition,"[/community/c1068820h2505927p21183575, /commun..."
4115,/community/c5389_2013_iran_team_selection_test,2013 Iran Team Selection Test,"[/community/c6h530094p3024260, /community/c6h5..."
4116,/community/c1241035_2018_saudi_arabia_imo_tst,2018 Saudi Arabia IMO TST,"[/community/c1068820h2210633p16725624, /commun..."


In [ ]:
all_links_df

,Unnamed: 0,category,contest,link
0,0,international_contests,IMO,/community/c3081085_2022imo_problems
1,1,international_contests,IMO,/community/c2413807_2021imo_problems
2,2,international_contests,IMO,/community/c1306546_2020imo
3,3,international_contests,IMO,/community/c912139_2019_imo
4,4,international_contests,IMO,/community/c681585_2018_imo
...,...,...,...,...
4149,4149,junior_olympiads,USAJMO,/community/c3977_2014_usajmo
4150,4150,junior_olympiads,USAJMO,/community/c3976_2013_usajmo
4151,4151,junior_olympiads,USAJMO,/community/c3975_2012_usajmo
4152,4152,junior_olympiads,USAJMO,/community/c3974_2011_usajmo


In [ ]:
df1 = all_links_df.drop_duplicates(subset='link')
df2 = total_mid_data_df.drop_duplicates(subset='link')

merged_df = pd.merge(df1, df2, on='link', how='inner')

In [ ]:
merged_df

,category,contest,link,name,source
0,international_contests,IMO,/community/c3081085_2022imo_problems,2022-IMO Problems,"[/community/c6h2883211p25635135, /community/c6..."
1,international_contests,IMO,/community/c2413807_2021imo_problems,2021-IMO Problems,"[/community/c6h2625909p22698392, /community/c6..."
2,international_contests,IMO,/community/c1306546_2020imo,2020-IMO,"[/community/c6h2278651p17821635, /community/c6..."
3,international_contests,IMO,/community/c912139_2019_imo,2019 IMO,"[/community/c6h1876068p12744859, /community/c6..."
4,international_contests,IMO,/community/c681585_2018_imo,2018 IMO,"[/community/c6h1670580p10626500, /community/c6..."
...,...,...,...,...,...
4000,junior_olympiads,USAJMO,/community/c3977_2014_usajmo,2014 USAJMO,"[/community/c5h587519p3477681, /community/c5h5..."
4001,junior_olympiads,USAJMO,/community/c3976_2013_usajmo,2013 USAJMO,"[/community/c5h532232p3041819, /community/c5h5..."
4002,junior_olympiads,USAJMO,/community/c3975_2012_usajmo,2012 USAJMO,"[/community/c5h476719p2669111, /community/c5h4..."
4003,junior_olympiads,USAJMO,/community/c3974_2011_usajmo,2011 USAJMO,"[/community/c5h404350p2254778, /community/c5h4..."


In [ ]:
merged_df.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
merged_df.to_csv('/content/drive/MyDrive/aops_complete_scraper/all_main_links.csv')

In [ ]:
exploded_main_links = merged_df.explode('source')

In [ ]:
exploded_main_links.reset_index(inplace=True)

In [ ]:
exploded_main_links.drop('index', inplace=True, axis=1)

In [ ]:
exploded_main_links.to_csv('/content/drive/MyDrive/aops_complete_scraper/all_main_links_exploded.csv')

In [ ]:
data = exploded_main_links

In [ ]:
data.head()

,category,contest,link,name,source
0,international_contests,IMO,/community/c3081085_2022imo_problems,2022-IMO Problems,/community/c6h2883211p25635135
1,international_contests,IMO,/community/c3081085_2022imo_problems,2022-IMO Problems,/community/c6h2883212p25635138
2,international_contests,IMO,/community/c3081085_2022imo_problems,2022-IMO Problems,/community/c6h2883213p25635143
3,international_contests,IMO,/community/c3081085_2022imo_problems,2022-IMO Problems,/community/c6h2883216p25635154
4,international_contests,IMO,/community/c3081085_2022imo_problems,2022-IMO Problems,/community/c6h2883217p25635158


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43300 entries, 0 to 43299
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   category  43300 non-null  object
 1   contest   43300 non-null  object
 2   link      43300 non-null  object
 3   name      43300 non-null  object
 4   source    43300 non-null  object
dtypes: object(5)
memory usage: 1.7+ MB


In [ ]:
len(data['source'].unique())

36828

In [ ]:
chunks = np.array_split(data, 16)

In [ ]:
chunks_name = [f'aops_part_{i}.csv' for i in range(1,17)]

In [ ]:
for i in range(len(chunks)):
  part = chunks[i]
  part.to_csv(f'/content/drive/MyDrive/aops_complete_scraper/{chunks_name[i]}')

In [ ]:
chunks_name

['aops_part_1.csv',
 'aops_part_2.csv',
 'aops_part_3.csv',
 'aops_part_4.csv',
 'aops_part_5.csv',
 'aops_part_6.csv',
 'aops_part_7.csv',
 'aops_part_8.csv',
 'aops_part_9.csv',
 'aops_part_10.csv',
 'aops_part_11.csv',
 'aops_part_12.csv',
 'aops_part_13.csv',
 'aops_part_14.csv',
 'aops_part_15.csv',
 'aops_part_16.csv']

In [ ]:
#Now run the code for all of the 16 parts making copies of this notebook

In [ ]:
#For Problem Number

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/aops_complete_scraper/aops_complete_data.csv')

In [ ]:
data.head()

,Unnamed: 0,category,contest,link,name,source,problem,problem_html,solution_1,solution_2,solution_3,solution_4,solution_5
0,0,international_contests,IMO,/community/c3081085_2022imo_problems,2022-IMO Problems,/community/c6h2883211p25635135,IMO 2022 Problem 1,"<div class=""cmty-post-html"">\n <div>\n </div>\...","[5, '<div class=""cmty-post-html"">\n <div>\n </...","[5, '<div class=""cmty-post-html"">\n <div>\n </...","[4, '<div class=""cmty-post-html"">\n <div>\n </...","[2, '<div class=""cmty-post-html"">\n <div>\n </...","[1, '<div class=""cmty-post-html"">\n <div>\n </..."
1,1,international_contests,IMO,/community/c3081085_2022imo_problems,2022-IMO Problems,/community/c6h2883212p25635138,IMO 2022 Problem 2,"<div class=""cmty-post-html"">\n <div>\n </div>\...","[11, '<div class=""cmty-post-html"">\n <div>\n <...","[6, '<div class=""cmty-post-html"">\n <div>\n </...","[6, '<div class=""cmty-post-html"">\n <div>\n </...","[5, '<div class=""cmty-post-html"">\n <div>\n </...","[3, '<div class=""cmty-post-html"">\n <div>\n </..."
2,2,international_contests,IMO,/community/c3081085_2022imo_problems,2022-IMO Problems,/community/c6h2883213p25635143,IMO 2022 Problem 3,"<div class=""cmty-post-html"">\n <div>\n </div>\...","[14, '<div class=""cmty-post-html"">\n <div>\n <...","[5, '<div class=""cmty-post-html"">\n <div>\n </...","[3, '<div class=""cmty-post-html"">\n <div>\n </...","[2, '<div class=""cmty-post-html"">\n <div>\n </...","[2, '<div class=""cmty-post-html"">\n <div>\n </..."
3,3,international_contests,IMO,/community/c3081085_2022imo_problems,2022-IMO Problems,/community/c6h2883216p25635154,IMO 2022 Problem 4,"<div class=""cmty-post-html"">\n <div>\n </div>\...","[10, '<div class=""cmty-post-html"">\n <div>\n <...","[4, '<div class=""cmty-post-html"">\n <div>\n </...","[3, '<div class=""cmty-post-html"">\n <div>\n </...","[2, '<div class=""cmty-post-html"">\n <div>\n </...","[2, '<div class=""cmty-post-html"">\n <div>\n </..."
4,4,international_contests,IMO,/community/c3081085_2022imo_problems,2022-IMO Problems,/community/c6h2883217p25635158,IMO 2022 Problem 5,"<div class=""cmty-post-html"">\n <div>\n </div>\...","[20, '<div class=""cmty-post-html"">\n <div>\n <...","[5, '<div class=""cmty-post-html"">\n <div>\n </...","[3, '<div class=""cmty-post-html"">\n <div>\n </...","[3, '<div class=""cmty-post-html"">\n <div>\n </...","[2, '<div class=""cmty-post-html"">\n <div>\n </..."


In [ ]:
link_data = data['link'].unique()

In [ ]:
len(link_data)

4005

In [ ]:
def get_link_and_name(url):
  post_class = ['cmty-view-posts-item','cmty-view-post-item-w-label','cmty-vp-both']
  link_class = 'cmty-view-post-poster'
  which_class= 'cmty-view-post-item-label'
  
  driver = driver_setup()

  wait = WebDriverWait(driver, 10)
  driver.get(domain+url)

  element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, f"div.{link_class}")))
  soup = BeautifulSoup(driver.page_source, 'html.parser')
  
  links_info = []
  posts = soup.find_all('div',{'class':post_class})

  for i in posts:
    try:
      link = i.find('div',{'class':link_class}).find('a').get('href')
      which = i.find('div',{'class':which_class}).text
      links_info.append([url,link,which])
    except:
      pass
  return links_info
  driver.quit()

In [ ]:
which_data = []
skipped_which = []
data = link_data
for i in data:
  index = int(np.argwhere(data==i))
  print(f'Progress: {round(index/len(data) * 100, 3)} % and left: {len(data) - index}')
  try:
    hehe = get_link_and_name(i)
    which_data.extend(hehe)
  except:
    skipped_which.append(i)

df = pd.DataFrame(which_data, columns=['link','source','which'])
df.to_csv('/content/drive/MyDrive/aops_complete_scraper/which_data.csv')

Progress: 0.0 % and left: 4005


<ipython-input-7-177052b956f3>:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("/usr/bin/chromedriver", options=options)


Progress: 0.025 % and left: 4004
Progress: 0.05 % and left: 4003
Progress: 0.075 % and left: 4002
Progress: 0.1 % and left: 4001
Progress: 0.125 % and left: 4000
Progress: 0.15 % and left: 3999
Progress: 0.175 % and left: 3998
Progress: 0.2 % and left: 3997
Progress: 0.225 % and left: 3996
Progress: 0.25 % and left: 3995
Progress: 0.275 % and left: 3994
Progress: 0.3 % and left: 3993
Progress: 0.325 % and left: 3992
Progress: 0.35 % and left: 3991
Progress: 0.375 % and left: 3990
Progress: 0.4 % and left: 3989
Progress: 0.424 % and left: 3988
Progress: 0.449 % and left: 3987
Progress: 0.474 % and left: 3986
Progress: 0.499 % and left: 3985
Progress: 0.524 % and left: 3984
Progress: 0.549 % and left: 3983
Progress: 0.574 % and left: 3982
Progress: 0.599 % and left: 3981
Progress: 0.624 % and left: 3980
Progress: 0.649 % and left: 3979
Progress: 0.674 % and left: 3978
Progress: 0.699 % and left: 3977
Progress: 0.724 % and left: 3976
Progress: 0.749 % and left: 3975
Progress: 0.774 % and 

In [ ]:
len(which_data)

42885

In [ ]:
len(skipped_which)

0

In [ ]:
df.tail()

,link,source,which
42880,/community/c4157_1997_turkey_junior_national_o...,/community/c6h517010p2908448,2
42881,/community/c4157_1997_turkey_junior_national_o...,/community/c6h517011p2908449,3
42882,/community/c4156_1996_turkey_junior_national_o...,/community/c6h517006p2908444,1
42883,/community/c4156_1996_turkey_junior_national_o...,/community/c6h517007p2908445,2
42884,/community/c4156_1996_turkey_junior_national_o...,/community/c6h517008p2908446,3


In [ ]:
df.sample(10)

,link,source,which
41448,/community/c4958_2002_china_team_selection_test,/community/c6h254142p1389564,2
26773,/community/c552450_2015_saint_petersburg_mathe...,/community/c6h1203701p5931980,6
26343,/community/c926757_1956_moscow_mathematical_ol...,/community/c893771h1899570p12981174,342
39131,/community/c3681_2009_sharygin_geometry_olympiad,/community/c6h279920p1511652,17
20032,/community/c2004666_1999_croatia_national_olym...,/community/c6h2564502p21988395,Problem 1
24194,/community/c3348_jbmo_tst__moldova,/community/c6h1824644p12199226,3
2292,/community/c3933_1982_imo_shortlist,/community/c6h366201p2014411,11
29557,/community/c1142186_2005_ukraine_national_math...,/community/c6h291919p1577436,7
15296,/community/c4352_2014_elmo_shortlist,/community/c6h599344p3557428,1
21898,/community/c3322_iran_mo_3rd_round,/community/c6h2679772p23237405,1


In [ ]:
df.loc[26343]['link']

'/community/c926757_1956_moscow_mathematical_olympiad'